In [1]:
# Loading all the important datasets used in q1
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

x_raw_data = np.load('./x_raw_data.npy')
y_raw_data = np.load('./y_raw_data.npy')
x_pca_data = np.load('./x_pca_data.npy')
y_pca_data = np.load('./y_pca_data.npy')
x_ten_data = np.load('./x_ten_data.npy')
y_ten_data = np.load('./y_ten_data.npy')
x_ten_pca_data = np.load('./x_ten_pca_data.npy')
y_ten_pca_data = np.load('./y_ten_pca_data.npy')

In [2]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(x_raw_data)

/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=10)

In [3]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
print(labels)

[1 7 7 ... 2 2 0]
(25000,)
9


In [21]:
closest_image = np.empty((0,3072))

for i in range(10):         # 10 clusters
    cluster_data_points = x_raw_data[labels == i]
    cluster_centroid = centroids[i]
    distances = np.linalg.norm(cluster_data_points - cluster_centroid,axis=1)
    closest_image = np.append(closest_image, [x_raw_data[np.argmin(distances)]], axis=0)

print(closest_image)

(10, 3072)


In [47]:
cluster_to_label = np.empty((0,))

for i in range(0,10):
    temp = y_raw_data[labels == i]
    mode = st.mode(temp).mode
    cluster_to_label = np.append(cluster_to_label,mode)

print(cluster_to_label)

[ 1.  1.  9. 10.  5.  8.  6.  1. 10.  5.]
